In [ ]:
!pip install dowhy

In [ ]:
!pip install psmpy

In [ ]:
%%time
import pandas as pd
import numpy as np
# import dowhy
from sklearn.linear_model import LinearRegression


CPU times: user 938 ms, sys: 137 ms, total: 1.07 s
Wall time: 2.91 s


In [ ]:
data_df = pd.read_pickle('/content/drive/MyDrive/Computational Social Systems/Master Thesis/headline_causal.pkl')
# data_df.drop(columns=['headline', 'clicks', 'clarification'], inplace=True)
# data_df['clarification'] = data_df['clarification'].map({'yes': 1, 'no': 0})


In [ ]:


list(data_df.columns)
data_df.headline.to_numpy()[0]


'Let’s See … Hire Cops, Pay Teachers, Buy Books For Schools. Or Kill People. Hard Choice, Right?'

index = (for all indices in embedding)

&nbsp;&nbsp;&nbsp;&nbsp;model = CTR ~ embedding[!index]

&nbsp;&nbsp;&nbsp;&nbsp;perPerson_perIndex_predicted_CTR = predict(model, for each person)

&nbsp;&nbsp;&nbsp;&nbsp;perPerson_group = embedding[index] >= median(embedding[index])


$R_i^2$ for each pair where one person is in each group, and predicted CTR is optimally close
   what is the actual CTR difference?

List of most predictive indices:

2, 145, 12...

In [ ]:
outcome_column = 'CTR'
predictor_columns = [col for col in data_df.columns if col != outcome_column]


for index_column in predictor_columns:

    predictors = [col for col in predictor_columns if col != index_column]

    model = LinearRegression()
    model.fit(data_df[predictors], data_df[outcome_column])

    data_df['predicted_CTR_' + index_column] = model.predict(data_df[predictors])

    median_embedding = data_df[index_column].median()
    data_df['perPerson_group_' + index_column] = data_df[index_column] >= median_embedding


In [ ]:
list(data_df.columns)
# data_new_df = pd.read_pickle('hello.pkl')


In [ ]:


# Step 3: Define a causal model
model = dowhy.CausalModel(
    data=data_df,
    treatment='formality_formal',  # Replace with the name of your treatment variable column
    outcome='CTR',      # Replace with the name of your outcome variable column
    common_causes=[
 'emotion_anger',
 'emotion_disgust',
 'emotion_fear',
 'emotion_joy',
 'emotion_neutral',
 'emotion_sadness',
 'emotion_surprise',
 'syntax_complexity',
 'basic_length',
 'giberish',
 'dale_chall_score',
 'sentiment_negative',
 'sentiment_neutral',
 'sentiment_positive']  # List other confounding variables
)

# Step 4: Identify causal estimands
identified_estimand = model.identify_effect()

# Step 5: Estimate causal effect using linear regression
estimate = model.estimate_effect(identified_estimand,
                                 method_name='backdoor.linear_regression')

# Print the causal effect estimate
print(estimate)


model.view_model()


In [ ]:
list(data.columns)

In [ ]:
data = pd.read_pickle('/content/drive/MyDrive/Computational Social Systems/Master Thesis/headline_causal_processed.pkl')
data.head(5)

,CTR,formality_formal,formality_informal,emotion_anger,emotion_disgust,emotion_fear,emotion_joy,emotion_neutral,emotion_sadness,emotion_surprise,...,predicted_CTR_embedding_380,perPerson_group_embedding_380,predicted_CTR_embedding_381,perPerson_group_embedding_381,predicted_CTR_embedding_382,perPerson_group_embedding_382,predicted_CTR_embedding_383,perPerson_group_embedding_383,predicted_CTR_embedding_384,perPerson_group_embedding_384
0,0.002566,0.998468,0.001532,0.409045,0.221184,0.063077,0.006619,0.250574,0.032195,0.017306,...,0.014411,False,0.014180,True,0.014519,False,0.014455,True,0.014379,True
1,0.028341,0.019113,0.980887,0.031717,0.085310,0.030770,0.007533,0.720395,0.076398,0.047876,...,0.021069,True,0.021140,True,0.021144,False,0.021086,False,0.021089,False
2,0.006298,0.035468,0.964532,0.342199,0.382035,0.093000,0.002185,0.131670,0.040235,0.008678,...,0.015430,False,0.015175,False,0.015451,False,0.015228,True,0.015363,False
3,0.008742,0.998651,0.001349,0.204931,0.533624,0.067115,0.001463,0.108103,0.064993,0.019770,...,0.012817,False,0.012809,False,0.012844,True,0.012904,True,0.012787,False
4,0.003279,0.964811,0.035189,0.316080,0.077622,0.040388,0.005667,0.213170,0.339549,0.007523,...,0.013880,False,0.013911,False,0.014059,False,0.013964,True,0.013878,False


In [ ]:
from dowhy import CausalModel

# df = pd.DataFrame(data)
def do_analysis(common_causes_main, df):
    for cause in common_causes_main:
        common_causes = [item for item in common_causes_main if item != cause]
        embedding_list = ['embedding_%d'%number for number in range(1,385)]
        common_causes.extend(embedding_list)

        causal_model = CausalModel(
            data=df,
            treatment='perPerson_group_'+cause,
            outcome='CTR',
            common_causes = common_causes,
            instruments=['predicted_CTR_'+cause]
        )

        # Identify the estimand
        identified_estimand = causal_model.identify_effect()

        # Estimate the treatment effect using propensity score matching
        estimate = causal_model.estimate_effect(
            identified_estimand,
            method_name="backdoor.propensity_score_matching",
            target_units="ate",
        )


        print(cause, '-', estimate.value)


In [ ]:
causes_list = ['formality_formal',
#  'formality_informal',
 'emotion_anger',
 'emotion_disgust',
 'emotion_fear',
 'emotion_joy',
 'emotion_neutral',
 'emotion_sadness',
 'emotion_surprise',
 'syntax_complexity',
 'basic_length',
 'giberish',
 'dale_chall_score',
 'sentiment_negative',
 'sentiment_neutral',
 'sentiment_positive']


do_analysis(causes_list, data)

In [ ]:
data[['formality_formal', 'predicted_CTR_formality_formal', 'perPerson_group_formality_formal']]

,formality_formal,predicted_CTR_formality_formal,perPerson_group_formality_formal
0,0.998468,0.014401,True
1,0.019113,0.021041,False
2,0.035468,0.015435,False
3,0.998651,0.012756,True
4,0.964811,0.013904,True
...,...,...,...
105546,0.998615,0.010407,True
105547,0.784095,0.019867,False
105548,0.675907,0.028118,False
105549,0.967737,0.015195,True


Effect

In [ ]:
embedding_list = ['embedding_%d'%number for number in range(1,385)]
np_array = data[embedding_list].head(20000).to_numpy()

In [ ]:
similarity = np.dot(np_array, np_array.T)

# squared magnitude of preference vectors (number of occurrences)
square_mag = np.diag(similarity)

# inverse squared magnitude
inv_square_mag = 1 / square_mag

# if it doesn't occur, set it's inverse magnitude to zero (instead of inf)
inv_square_mag[np.isinf(inv_square_mag)] = 0

# inverse of the magnitude
inv_mag = np.sqrt(inv_square_mag)

# cosine similarity (elementwise multiply by inverse magnitudes)
cosine = similarity * inv_mag
cosine = cosine.T * inv_mag

cosine.shape

(20000, 20000)

In [ ]:
formality_formal -  -0.0009003530568299572
emotion_anger -      0.0010072101737743074
emotion_disgust -    0.002301255130395963
emotion_fear -       0.001198326734492035
emotion_joy -       -0.000985384507931537
emotion_neutral -   -0.002278599202807139
emotion_sadness -    0.0014750827003031533
emotion_surprise -  -0.0005571921426504084
syntax_complexity -  0.0017304779548953698
basic_length -      -0.0017429606553956436
giberish -           0.00033531771473939014
dale_chall_score -   0.0017182004012356183
sentiment_negative - 0.0022922630018918674
sentiment_neutral - -0.0012223326278845113
sentiment_positive- -0.0021200611400042342



NameError: ignored

-----------------------------------------------------------------------------

In [ ]:
!pip install nougat-ocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.2/80.2 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.5/431.5 kB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 96.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 93.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 104.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.0/660.0 kB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install tensorrt

  Preparing metadata (setup.py) ... done
  Created wheel for tensorrt: filename=tensorrt-8.6.1.post1-py2.py3-none-any.whl size=17281 sha256=0a38ba23a5c2459f9a8e272fe2d11f68cc982ac8627b1ed1007a95d0eae92f36
  Stored in directory: /root/.cache/pip/wheels/f4/c8/0e/b79b08e45752491b9acfdbd69e8a609e8b2ed7640dda5a3e59
Successfully built tensorrt


In [ ]:

%%time
!nougat /content/common_sense.pdf -o output_directory

2023-09-19 16:42:31.152389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
  0% 0/14 [00:00<?, ?it/s]INFO:root:Processing file /content/common_sense.pdf with 54 pages
100% 14/14 [02:06<00:00,  9.01s/it]
CPU times: user 857 ms, sys: 120 ms, total: 978 ms
Wall time: 2min 23s
